# IMC-DIMR and marker-individual exports

Hot pixel removal by removing local maxima in adjacent 3x3 sliding windows and export by marker

In [1]:
import sys
sys.path.append("/home/daria_l/src/MapMet/code/python/Image_Preprocessing/IMC-Denoise_adapted")

In [2]:
import IMC_Denoise
from IMC_Denoise.IMC_Denoise_main.DIMR import DIMR
import pandas as pd
import os, glob
import tifffile

After spillover compensation in R, we first have to convert the images to float32 as they are written from R in uint32. And we also have to attach channel names to the ome-tiffs.

In [18]:
import os
import tifffile
import pandas as pd
import glob
from xtiff import to_tiff
import numpy as np

base_path = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria"
samples = ["20220907_10-1095_TU"]
#samples = os.listdir(os.path.join(base_path, "TU"))

out_path = os.path.join(base_path, "_Data_Analysis/_tmp_daria/Image_analysis/20221102_Steinbock_Mesmer_IFbased/PT")
marker_codes = pd.read_csv(os.path.join(out_path, 'marker_codes.csv'), sep=";")

for s in samples:
    mc_image_files = glob.glob(os.path.join(out_path + "/comp_img/{}*.tiff".format(s)), recursive=True)

    if not os.path.exists(os.path.join(base_path, "TU", s, "IF_IMC_comp")):
        os.mkdir(os.path.join(base_path, "TU", s, "IF_IMC_comp"))

    for f in mc_image_files:
        img = tifffile.imread(f)
        img= img.astype(np.float32)

        to_tiff(
            img,
            os.path.join(base_path, "TU", s, "IF_IMC_comp", f.split("/")[-1]),
            image_name=os.path.join(base_path, "TU", s, "IF_IMC_comp", f.split("/")[-1]),
            channel_names=list(marker_codes["new_names"]),
            pixel_size=1.0,
            pixel_depth=1.0,
            )

/home/daria_l/anaconda3/envs/mapmet/lib/python3.10/site-packages/xtiff/tiff.py:130: UserWarning: The specified OME-TIFF file name does not end with .ome.tiff: 20220907_10-1095_TU_ROI_003_IF_IMC.tiff
  warnings.warn(
/home/daria_l/anaconda3/envs/mapmet/lib/python3.10/site-packages/xtiff/tiff.py:130: UserWarning: The specified OME-TIFF file name does not end with .ome.tiff: 20220907_10-1095_TU_ROI_002_IF_IMC.tiff
  warnings.warn(
/home/daria_l/anaconda3/envs/mapmet/lib/python3.10/site-packages/xtiff/tiff.py:130: UserWarning: The specified OME-TIFF file name does not end with .ome.tiff: 20220907_10-1095_TU_ROI_004_IF_IMC.tiff
  warnings.warn(
/home/daria_l/anaconda3/envs/mapmet/lib/python3.10/site-packages/xtiff/tiff.py:130: UserWarning: The specified OME-TIFF file name does not end with .ome.tiff: 20220907_10-1095_TU_ROI_001_IF_IMC.tiff
  warnings.warn(


Delete below part later. This is just for when the training samples are already randomly selected.

In [8]:
# import os
# import tifffile
# import xml.etree.ElementTree
# import pandas as pd
# import io
# import random
# import glob
# import numpy as np

# def read_channel_names(path):
    
#     tiff = tifffile.TiffFile(path)
#     omexml_string = tiff.pages[0].description
#     root = xml.etree.ElementTree.parse(io.StringIO(omexml_string))
#     namespaces = {'ome': 'http://www.openmicroscopy.org/Schemas/OME/2016-06'}
#     channels = root.findall('ome:Image[1]/ome:Pixels/ome:Channel', namespaces)
#     channel_names = [c.attrib['Name'] for c in channels]
#     return channel_names


# base_path = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria"

# BM_path = os.path.join(base_path, "BM")
# out_path = os.path.join(base_path, "_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik")

# training_samples = os.listdir(os.path.join(out_path, "training", "raw"))

# for s in training_samples: 

#     sample = s.split("_ROI")[0]
#     training_file = s.split(".")[0]

#     multichannel_path = os.path.join(BM_path, sample, "IF_IMC_comp_196")

#     mc_image_files = os.listdir(multichannel_path)

#     for f in mc_image_files:
#         file_name = f.split(".")[0]

#         mc_image = tifffile.imread(os.path.join(multichannel_path, f))

#         channel_names = read_channel_names(os.path.join(multichannel_path, f))

#         if file_name == training_file:
#             folder = "training"
#         else:
#             folder = "other"
        
#         if not os.path.exists(os.path.join(out_path, folder, "comp_196", file_name)):
#             os.mkdir(os.path.join(out_path, folder, "comp_196", file_name))

#         for c, img in enumerate(mc_image):
#             tifffile.imwrite(os.path.join(out_path, folder, "comp_196", file_name, channel_names[c]+".tiff"),img)


After spillover compensation, we have to prepare the file structure that is required by IMC-Denoise for DIMR hot-pixel removal.

In [7]:
import os
import tifffile
import xml.etree.ElementTree
import pandas as pd
import io
import random
import glob
import numpy as np

def read_channel_names(path):
    
    tiff = tifffile.TiffFile(path)
    omexml_string = tiff.pages[0].description
    root = xml.etree.ElementTree.parse(io.StringIO(omexml_string))
    namespaces = {'ome': 'http://www.openmicroscopy.org/Schemas/OME/2016-06'}
    channels = root.findall('ome:Image[1]/ome:Pixels/ome:Channel', namespaces)
    channel_names = [c.attrib['Name'] for c in channels]
    return channel_names


base_path = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria"

BM_path = os.path.join(base_path, "TU")
out_path = os.path.join(base_path, "_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik/PT")

if not os.path.exists(os.path.join(out_path, "training")):
    os.mkdir(os.path.join(out_path, "training"))

if not os.path.exists(os.path.join(out_path, "other")):
    os.mkdir(os.path.join(out_path, "other"))

samples = os.listdir(BM_path)

for s in samples: 

    multichannel_path = os.path.join(BM_path, s, "IF_IMC_comp")

    mc_image_files = os.listdir(multichannel_path)

    training_file = random.choice(mc_image_files)
    training_file = training_file.split(".")[0]


    for f in mc_image_files:
        file_name = f.split(".")[0]

        mc_image = tifffile.imread(os.path.join(multichannel_path, f))

        channel_names = read_channel_names(os.path.join(multichannel_path, f))

        if file_name == training_file:
            folder = "training"
        else:
            folder = "other"

        if not os.path.exists(os.path.join(out_path, folder, "comp", file_name)):
            os.mkdir(os.path.join(out_path, folder, "comp", file_name))

        for c, img in enumerate(mc_image):
            tifffile.imwrite(os.path.join(out_path, folder, "comp", file_name, channel_names[c]+".tiff"),img)


After we have generated single images out of the stacks for each RoI of each sample, we can apply DIMR hot-pixel removal. We will randomly select one RoI per sample for training of DIMR and Ilastik background prediction. 

In [ ]:
base_path = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik/PT"


marker_codes = pd.read_csv(os.path.join(base_path, 'marker_codes.csv'), sep=";")

n_neighbours = 4 # Larger n enables removing more consecutive hot pixels. 
n_iter = 3 # Iteration number for DIMR
window_size = 3 # Slide window size. For IMC images, window_size = 3 is fine.

other_path = os.path.join(base_path, "other")
training_path = os.path.join(base_path, "training")

for m in marker_codes["new_names"]:

    if m in ["131Xe-crtl1_Xe131", "134Xe-crtl2_Xe134", "136Ba-crtl3_Ba136"]:
        continue

    working_path = training_path

    if not os.path.exists(os.path.join(working_path, "DIMR", m)):
        os.mkdir(os.path.join(working_path, "DIMR", m))

    if not os.path.exists(os.path.join(working_path, "DIMR_Ilastik", m)):
        os.mkdir(os.path.join(working_path, "DIMR_Ilastik", m))

    for s in glob.glob(os.path.join(working_path + "/comp/**/*{}.tiff".format(m)), recursive=True):
        img_raw = tifffile.imread(s)
        img_DIMR = DIMR(n_neighbours = n_neighbours, n_iter = n_iter, window_size = window_size).perform_DIMR(img_raw)
        tifffile.imwrite(os.path.join(working_path, "DIMR", m, s.split("/")[-2].split("_IF")[0] + "_" + s.split("/")[-1]), img_DIMR)


    working_path = other_path

    if not os.path.exists(os.path.join(working_path, "DIMR", m)):
        os.mkdir(os.path.join(working_path, "DIMR", m))

    if not os.path.exists(os.path.join(working_path, "DIMR_Ilastik", m)):
        os.mkdir(os.path.join(working_path, "DIMR_Ilastik", m))

    for s in glob.glob(os.path.join(working_path + "/comp/**/*{}.tiff".format(m)), recursive=True):
        img_raw = tifffile.imread(s)
        img_DIMR = DIMR(n_neighbours = n_neighbours, n_iter = n_iter, window_size = window_size).perform_DIMR(img_raw)
        tifffile.imwrite(os.path.join(working_path, "DIMR", m, s.split("/")[-2].split("_IF")[0] + "_" + s.split("/")[-1]), img_DIMR)
    

After DIMR correction, we will perform background prediciton in Ilastik and save the inferred masks. In a next step, we will go through all samples and per marker read in the comp-DIMR corrected images and ilastik masks of all RoI. In case of BM samples, if the marker is an IF marker or CD274, we will set the entire mask to one, since we did not perform fg/bg prediction for those markers.

In [ ]:
import os
import glob
import pandas as pd
from functools import reduce
import numpy as np 
import tifffile
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from xtiff import to_tiff
import re

base_path = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria"
BM_path = os.path.join(base_path, "TU")
samples = os.listdir(BM_path)

path_to_data = os.path.join(base_path, "_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik/PT")
marker_codes = pd.read_csv(os.path.join(path_to_data, 'marker_codes.csv'), sep=";")
marker_codes = [i for i in list(marker_codes.new_names) if i not in ["131Xe-crtl1_Xe131", "134Xe-crtl2_Xe134", "136Ba-crtl3_Ba136"]]

def merge_on_marker(folder:str, sample: str, data_path:str) -> pd.DataFrame:

    roi = os.listdir(os.path.join(BM_path, sample, "IF_IMC_lowres"))
    df_list = []

    for _r in roi:
        r = _r.split("_IF")[0]
        paths = glob.glob(os.path.join(data_path + "/*/" + folder + "/*/{}*.tiff".format(r)), recursive=True)
        marker = [i.split("/")[-2] for i in paths]
        df = pd.DataFrame({"marker": marker, r: paths})
        df_list.append(df)

    merged_df = reduce(lambda df1,df2: pd.merge(df1,df2,on="marker", how="outer"), df_list)
    merged_df.set_index("marker", inplace=True)

    return merged_df

def stack_save(df: pd.DataFrame, out_path: str):
    for _roi in df.columns:
        mc_image = np.stack(df[_roi])
        ch_names = df.index

        to_tiff(
            mc_image,
            os.path.join(out_path, _roi + "_IF_IMC.tiff"),
            image_name=os.path.join(out_path, _roi + "_IF_IMC.tiff"),
            channel_names=list(ch_names),
            pixel_size=1.0,
            pixel_depth=1.0,
            )


for s in samples: 
    if not os.path.exists(os.path.join(BM_path, s, "IF_IMC_processed")):
        os.mkdir(os.path.join(BM_path, s, "IF_IMC_processed"))

    if not os.path.exists(os.path.join(BM_path, s, "IF_IMC_ilastik-masks")):
        os.mkdir(os.path.join(BM_path, s, "IF_IMC_ilastik-masks"))
    
    img_paths_df = merge_on_marker("DIMR", s, path_to_data)
    mask_paths_df = merge_on_marker("DIMR_ilastik_simplesegmentation", s, path_to_data)

    mask_paths_df = mask_paths_df.loc[:, img_paths_df.columns]

    img_df = img_paths_df.loc[marker_codes, :]
    mask_df = img_paths_df.loc[marker_codes, :]

    for m in marker_codes:
        img_paths = img_paths_df.loc[m]
        imgs = np.array([tifffile.imread(f) for f in img_paths])

        #if m in ["1IF_DAPI_IF1", "2IF_GD2-IF2", "3IF_CD56-IF3", "160Gd-CD274_Gd160"]:
        if m in ["1IF_DAPI_IF1", "2IF_GD2-IF2", "3IF_CD56-IF3"]:
            masks = np.ones_like(imgs)
        else:
            mask_paths = mask_paths_df.loc[m]
            masks = np.array([tifffile.imread(f) for f in mask_paths])


        imgs_cut = imgs[:, 2:-2, 2:-2]
        masks_cut = masks[:, 2:-2, 2:-2]

        #masks_cut[masks_cut==2] = 0

        imgs_99 = imgs_cut.copy()

        # only for BM samples: as HLA-ABC has a lot of outliers we will set a lower upper threshold to make sure that the "true" signals are not squeezed
        '''if m == "147Sm-HLA-ABC_Sm147":
            imgs_99[imgs_99 > np.percentile(imgs_99, 99)] = np.percentile(imgs_99, 99)
        else:
            imgs_99[imgs_99 > np.percentile(imgs_99, 99.5)] = np.percentile(imgs_99, 99.5)'''
        
        imgs_99[imgs_99 > np.percentile(imgs_99, 99.5)] = np.percentile(imgs_99, 99.5)

        imgs_wo_bg = imgs_99 * masks_cut

        scaler = MinMaxScaler()
        tmp = imgs_wo_bg.reshape(-1, 1)
        tmp_norm = scaler.fit_transform(tmp)
        imgs_norm = tmp_norm.reshape(imgs_wo_bg.shape)

        img_df.loc[m] = [i for i in imgs_norm]
        mask_df.loc[m] = [i for i in masks_cut]

    stack_save(img_df, os.path.join(BM_path, s, "IF_IMC_processed"))
    stack_save(mask_df, os.path.join(BM_path, s, "IF_IMC_ilastik-masks"))

    print("Sample " + s + " finished and saved.")


After exporting the processed images and ilastik masks, we will to clipping and normalization on the IF images and copy them to the probabilites folder. 

In [20]:
import os
import glob
import pandas as pd
from functools import reduce
import numpy as np 
import tifffile
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from xtiff import to_tiff
import re

base_path = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria"
BM_path = os.path.join(base_path, "TU")
samples = os.listdir(BM_path)

marker_codes = ["1IF_DAPI_IF1", "2IF_GD2-IF2", "3IF_CD56-IF3"]

path_to_data = os.path.join(base_path, "_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik/PT")

def merge_on_marker(folder:str, sample: str, data_path:str) -> pd.DataFrame:

    roi = os.listdir(os.path.join(BM_path, sample, "IF_IMC_lowres"))
    df_list = []

    for _r in roi:
        r = _r.split("_IF")[0]
        paths = glob.glob(os.path.join(data_path + "/*/" + folder + "/*/{}*.tiff".format(r)), recursive=True)
        marker = [i.split("/")[-2] for i in paths]
        df = pd.DataFrame({"marker": marker, r: paths})
        df_list.append(df)

    merged_df = reduce(lambda df1,df2: pd.merge(df1,df2,on="marker", how="outer"), df_list)
    merged_df.set_index("marker", inplace=True)

    return merged_df

for s in samples: 
    
    img_paths_df = merge_on_marker("DIMR", s, path_to_data)

    img_paths_df = img_paths_df.loc[marker_codes, :]

    for m in marker_codes:
        img_paths = img_paths_df.loc[m]
        imgs = np.array([tifffile.imread(f) for f in img_paths])

        imgs_99 = imgs.copy()
        
        imgs_99[imgs_99 > np.percentile(imgs_99, 99.5)] = np.percentile(imgs_99, 99.5)

        scaler = MinMaxScaler()
        tmp = imgs_99.reshape(-1, 1)
        tmp_norm = scaler.fit_transform(tmp)
        imgs_norm = tmp_norm.reshape(imgs_99.shape)

        img_paths_probs = [p.replace("/DIMR/", "/DIMR_ilastik_probabilities/") for p in img_paths]
        [tifffile.imwrite(img_paths_probs[i], imgs_norm[i]) for i in range(len(imgs_norm))]

    print("Sample " + s + " finished and saved.")

Sample 20220907_05-3231_TU finished and saved.
Sample 20220809_10-2679_TU finished and saved.
Sample 20220804_18-1061_TU finished and saved.
Sample 20220809_08-4160_TU finished and saved.
Sample 20220810_05-0034_TU finished and saved.
Sample 20220825_03-0313_TU finished and saved.
Sample 20220810_05-3938_TU finished and saved.
Sample 20220810_09-0411_TU finished and saved.
Sample 20220809_07-4158_TU finished and saved.
Sample 20220816_11-1208_TU finished and saved.
Sample 20220804_16-0274_TU finished and saved.
Sample 20220809_16-1934_TU finished and saved.
Sample 20220906_03-1501_TU finished and saved.
Sample 20220906_03-3536_TU finished and saved.
Sample 20220907_10-1095_TU finished and saved.
Sample 20220906_03-0170_TU finished and saved.
Sample 20220804_17-2178_TU finished and saved.
Sample 20220906_02-4010_TU finished and saved.
Sample 20220825_02-2222_TU finished and saved.
Sample 20220908_21-00438_TU finished and saved.
Sample 20220908_21-00560_TU finished and saved.
Sample 2022

Now we will stack all probabilities that were exported in Ilastik and save them to the original sample folders.

In [3]:
import os
import glob
import pandas as pd
from functools import reduce
import numpy as np 
import tifffile
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from xtiff import to_tiff
import re

base_path = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria"
BM_path = os.path.join(base_path, "TU")
samples = os.listdir(BM_path)

path_to_data = os.path.join(base_path, "_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik/PT")
marker_codes = pd.read_csv(os.path.join(path_to_data, 'marker_codes.csv'), sep=";")
marker_codes = [i for i in list(marker_codes.new_names) if i not in ["131Xe-crtl1_Xe131", "134Xe-crtl2_Xe134", "136Ba-crtl3_Ba136"]]

def merge_on_marker(folder:str, sample: str, data_path:str) -> pd.DataFrame:

    roi = os.listdir(os.path.join(BM_path, sample, "IF_IMC_lowres"))
    df_list = []

    for _r in roi:
        r = _r.split("_IF")[0]
        paths = glob.glob(os.path.join(data_path + "/*/" + folder + "/*/{}*.tiff".format(r)), recursive=True)
        marker = [i.split("/")[-2] for i in paths]
        df = pd.DataFrame({"marker": marker, r: paths})
        df_list.append(df)

    merged_df = reduce(lambda df1,df2: pd.merge(df1,df2,on="marker", how="outer"), df_list)
    merged_df.set_index("marker", inplace=True)

    return merged_df

def stack_save(df: pd.DataFrame, out_path: str):
    for _roi in df.columns:
        mc_image = np.stack(df[_roi])
        ch_names = df.index

        to_tiff(
            mc_image,
            os.path.join(out_path, _roi + "_IF_IMC.tiff"),
            image_name=os.path.join(out_path, _roi + "_IF_IMC.tiff"),
            channel_names=list(ch_names),
            pixel_size=1.0,
            pixel_depth=1.0,
            )


for s in samples: 

    if not os.path.exists(os.path.join(BM_path, s, "IF_IMC_ilastik-probabilities")):
        os.mkdir(os.path.join(BM_path, s, "IF_IMC_ilastik-probabilities"))
    
    img_paths_df = merge_on_marker("DIMR_ilastik_probabilities", s, path_to_data)

    img_paths_df = img_paths_df.loc[marker_codes, :]

    for m in marker_codes:
        img_paths = img_paths_df.loc[m]
        imgs = np.array([tifffile.imread(f) for f in img_paths])

        imgs_cut = imgs[:, 2:-2, 2:-2]

        img_paths_df.loc[m] = [i for i in imgs_cut]

    stack_save(img_paths_df, os.path.join(BM_path, s, "IF_IMC_ilastik-probabilities"))

    print("Sample " + s + " finished and saved.")

/home/daria_l/anaconda3/envs/mapmet/lib/python3.10/site-packages/xtiff/tiff.py:130: UserWarning: The specified OME-TIFF file name does not end with .ome.tiff: 20220907_05-3231_TU_ROI_001_IF_IMC.tiff
  warnings.warn(


Sample 20220907_05-3231_TU finished and saved.


/home/daria_l/anaconda3/envs/mapmet/lib/python3.10/site-packages/xtiff/tiff.py:130: UserWarning: The specified OME-TIFF file name does not end with .ome.tiff: 20220809_10-2679_TU_ROI_006_IF_IMC.tiff
  warnings.warn(
/home/daria_l/anaconda3/envs/mapmet/lib/python3.10/site-packages/xtiff/tiff.py:130: UserWarning: The specified OME-TIFF file name does not end with .ome.tiff: 20220809_10-2679_TU_ROI_004_IF_IMC.tiff
  warnings.warn(
/home/daria_l/anaconda3/envs/mapmet/lib/python3.10/site-packages/xtiff/tiff.py:130: UserWarning: The specified OME-TIFF file name does not end with .ome.tiff: 20220809_10-2679_TU_ROI_005_IF_IMC.tiff
  warnings.warn(
/home/daria_l/anaconda3/envs/mapmet/lib/python3.10/site-packages/xtiff/tiff.py:130: UserWarning: The specified OME-TIFF file name does not end with .ome.tiff: 20220809_10-2679_TU_ROI_001_IF_IMC.tiff
  warnings.warn(


Sample 20220809_10-2679_TU finished and saved.


/home/daria_l/anaconda3/envs/mapmet/lib/python3.10/site-packages/xtiff/tiff.py:130: UserWarning: The specified OME-TIFF file name does not end with .ome.tiff: 20220804_18-1061_TU_ROI_008_IF_IMC.tiff
  warnings.warn(
/home/daria_l/anaconda3/envs/mapmet/lib/python3.10/site-packages/xtiff/tiff.py:130: UserWarning: The specified OME-TIFF file name does not end with .ome.tiff: 20220804_18-1061_TU_ROI_005_IF_IMC.tiff
  warnings.warn(
/home/daria_l/anaconda3/envs/mapmet/lib/python3.10/site-packages/xtiff/tiff.py:130: UserWarning: The specified OME-TIFF file name does not end with .ome.tiff: 20220804_18-1061_TU_ROI_009_IF_IMC.tiff
  warnings.warn(
/home/daria_l/anaconda3/envs/mapmet/lib/python3.10/site-packages/xtiff/tiff.py:130: UserWarning: The specified OME-TIFF file name does not end with .ome.tiff: 20220804_18-1061_TU_ROI_003_IF_IMC.tiff
  warnings.warn(


...

Sample 20220907_13-3719_TU finished and saved.


/home/daria_l/anaconda3/envs/mapmet/lib/python3.10/site-packages/xtiff/tiff.py:130: UserWarning: The specified OME-TIFF file name does not end with .ome.tiff: 20220908_20-2831_TU_ROI_001_IF_IMC.tiff
  warnings.warn(
/home/daria_l/anaconda3/envs/mapmet/lib/python3.10/site-packages/xtiff/tiff.py:130: UserWarning: The specified OME-TIFF file name does not end with .ome.tiff: 20220908_20-2831_TU_ROI_002_IF_IMC.tiff
  warnings.warn(
/home/daria_l/anaconda3/envs/mapmet/lib/python3.10/site-packages/xtiff/tiff.py:130: UserWarning: The specified OME-TIFF file name does not end with .ome.tiff: 20220908_20-2831_TU_ROI_004_IF_IMC.tiff
  warnings.warn(
/home/daria_l/anaconda3/envs/mapmet/lib/python3.10/site-packages/xtiff/tiff.py:130: UserWarning: The specified OME-TIFF file name does not end with .ome.tiff: 20220908_20-2831_TU_ROI_003_IF_IMC.tiff
  warnings.warn(


Sample 20220908_20-2831_TU finished and saved.


/home/daria_l/anaconda3/envs/mapmet/lib/python3.10/site-packages/xtiff/tiff.py:130: UserWarning: The specified OME-TIFF file name does not end with .ome.tiff: 20220809_13-4453_TU_ROI_005_IF_IMC.tiff
  warnings.warn(
/home/daria_l/anaconda3/envs/mapmet/lib/python3.10/site-packages/xtiff/tiff.py:130: UserWarning: The specified OME-TIFF file name does not end with .ome.tiff: 20220809_13-4453_TU_ROI_003_IF_IMC.tiff
  warnings.warn(


Sample 20220809_13-4453_TU finished and saved.
